# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116958e+02     1.500496e+00
 * time: 0.8037819862365723
     1     9.650855e+00     8.922791e-01
 * time: 2.0646259784698486
     2    -1.157081e+01     1.037945e+00
 * time: 2.657961845397949
     3    -3.392960e+01     7.334544e-01
 * time: 3.4649078845977783
     4    -4.741695e+01     4.991691e-01
 * time: 4.25744891166687
     5    -5.697837e+01     2.147135e-01
 * time: 5.087173938751221
     6    -5.987778e+01     1.590732e-01
 * time: 5.620847940444946
     7    -6.092916e+01     4.525865e-02
 * time: 6.145229816436768
     8    -6.132460e+01     6.666359e-02
 * time: 6.648605823516846
     9    -6.156191e+01     3.737103e-02
 * time: 7.179724931716919
    10    -6.176768e+01     2.884225e-02
 * time: 7.685828924179077
    11    -6.193426e+01     2.691123e-02
 * time: 8.222980976104736
    12    -6.200957e+01     1.681799e-02
 * time: 8.762418985366821
    13    -6.208148e+01     1.574938e-02
 * time: 9.293834924697876
    1

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671082
    AtomicLocal         -18.8557744
    AtomicNonlocal      14.8522683
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485396 
    Xc                  -19.3336829

    total               -62.261666455814